In [1]:
%matplotlib inline
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [7]:
##############################################################
# CLASS TO NORMALIZE OUR DATA
##############################################################

class Normalization(object):
    def __init__(self, mean, std):
        self.mean = mean.view(-1, 1, 1)
        self.std = std.view(-1, 1, 1)
    def __call__(self, sample):
        image, label1, label2, label3 = sample['image'],\
        sample['label_age'], sample['label_gender'], sample['label_race']
        
        return {'image': image,
                'label_age': label1,
                'label_gender': label2,
                'label_race': label3}

##############################################################
# CLASS FOR DATA HANDLING IN PYTORCH
##############################################################

class MyData(Dataset):
    def __init__(self, dataset_tensor, train=True, transform=None):
        
        #Leaving only image related columns
        features=dataset_tensor[:,:39999]
     
        ##########################################################
        # Setting labels
        ##########################################################
       
        label_chest_circumference=dataset_tensor[:,40000]
        label_height=dataset_tensor[:,40001]
        label_inseam=dataset_tensor[:,40002]
        label_left_arm_length=dataset_tensor[:,40003]
        label_pelvis_circumference=dataset_tensor[:,40004]
        label_right_arm_length=dataset_tensor[:,40005]
        label_shoulder_width=dataset_tensor[:,40006]
        label_waist_circumference=dataset_tensor[:,40007]
        
        ##########################################################
        # splitting the data into train and validation set
        ##########################################################
        
        X_train, X_test, y_chest_circumference_train, y_chest_circumference_test,\
        y_height_train, y_height_test, y_inseam_train, y_inseam_test,\
        y_left_arm_length_train, y_left_arm_length_test,\
        y_pelvis_circumference_train, y_pelvis_circumference_test,\
        y_right_arm_length_train, y_right_arm_length_test,\
        y_shoulder_width_train, y_shoulder_width_test,\
        y_waist_circumference_train, y_waist_circumference_test = train_test_split(features, label_chest_circumference, 
                           label_height, label_inseam,
                           label_left_arm_length, label_pelvis_circumference,
                           label_right_arm_length, label_shoulder_width,
                           label_waist_circumference, test_size=0.2)
        
        if train==True:
            self.x=X_train
            self.chest_circumference_y=y_chest_circumference_train
            self.height_y=y_height_train
            self.inseam_y=y_inseam_train
            self.left_arm_length_y=y_left_arm_length_train
            self.pelvis_circumference_y=y_pelvis_circumference_train
            self.right_arm_length_y=y_right_arm_length_train
            self.shoulder_width_y=y_shoulder_width_train
            self.waist_circumference_y=y_waist_circumference_train
        else:
            self.x=X_test
            self.chest_circumference_y=y_chest_circumference_test
            self.height_y=y_height_test
            self.inseam_y=y_inseam_test
            self.left_arm_length_y=y_left_arm_length_test
            self.pelvis_circumference_y=y_pelvis_circumference_test
            self.right_arm_length_y=y_right_arm_length_test
            self.shoulder_width_y=y_shoulder_width_test
            self.waist_circumference_y=y_waist_circumference_test 
            
        #############################
        # TRANSFORMS
        #############################
        
        # normalize data, w.o. Bessel Correction -> n instead of n-1
        std, mean = torch.std_mean(self.x, unbiased=False)
        
        if transform is None:
            intrinsic_transform = torch.nn.Sequential(transforms.Normalize(mean, std))
        
            #Applying transformation
            self.transform=intrinsic_transform
        else:
            #Applying transformation
            self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=np.array(self.x[idx, 0:]).astype('float')
        label1=np.array([self.chest_circumference_y[idx]]).astype('float')
        label2=np.array([self.height_y[idx]]).astype('float')
        label3=np.array([self.inseam_y[idx]]).astype('float')
        label4=np.array([self.left_arm_length_y[idx]]).astype('float')
        label5=np.array([self.pelvis_circumference_y[idx]]).astype('float')
        label6=np.array([self.right_arm_length_y[idx]]).astype('float')
        label7=np.array([self.shoulder_width_y[idx]]).astype('float')
        label8=np.array([self.waist_circumference_y[idx]]).astype('float')
        
        sample={'image': np.uint8(image), 'label_chest_circumference': label1,
                'label_height': label2, 'label_inseam': label3, 'label_left_arm_length': label4,
                'label_pelvis_circumference': label5, 'label_.right_arm_length': label6, 'label_shoulder_width': label7,
                'label_waist_circumference': label8}
        
        #Applying transformation
        if self.transform:
            sample=self.transform(sample)
            
        return sample

In [8]:
##############################################################
# LOAD TENSORS FROM FILES
##############################################################

device = torch.device('cuda')

plain_tensor = torch.load('tensors/humans_monsters_plain_full_tensor.pt', map_location=device)
#rgb_tensor = torch.load('tensors/humans_monsters_rgb_full_tensor.pt', map_location=device)
#texture_tensor = torch.load('tensors/humans_monsters_texture_full_tensor.pt', map_location=device)

##############################################################
# CREATE DATA OBJECTS AND PREPARE THEM FOR THE CNN WITH
# THE DATA LOADER CLASS
##############################################################

plain_dataset = MyData(plain_tensor)
#rgb_dataset = MyData(rgb_tensor)
#texture_dataset = MyData(texture_tensor)
                       
loader = DataLoader(plain_dataset, batch_size=10, num_workers=0, shuffle=False)
#loader = DataLoader(plain_dataset, batch_size=10, num_workers=0, shuffle=False)
#loader = DataLoader(plain_dataset, batch_size=10, num_workers=0, shuffle=False)
                       
print("Dataset/s loaded")

Dataset loaded


#### Original Architecture

* Input Layer: fixed image of size 200 x 200 x 1
* Second Layer: Convolution with 5-pixels square kernel. Output: feature map of size 196 x 196 x 8.

* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Pooling Layer: Max pooling with stride 2 
* Convolution Layer: Convolution with 5-pixels square kernel and 16 output channels Output: Tensor of size 94 x 94 x 16.

* Pooling Layer: Max pooling with stride 2. And then flatten. Output: tensor of size 3544

* Fully Connected Layer:
* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Regressor Layer: Output: 8 human body dimensions in meters

#### Training
* 20 epochs
* mini_batc_size = 100
* Loss = MSE
* learning_rate = 0.01
* momentum = 0.9
* TODO: regressor funktion
* For layers
* 
* https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html

In [ ]:
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 8, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=1),
        
      nn.Conv2d(8, 16, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=1),
        
      nn.Flatten(),
      nn.Linear(),
        
      nn.ReLU()
    )


    
nn.
  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
  
    
    
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First 2D convolutional layer, taking in 1 input channel (image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      # Second 2D convolutional layer, taking in the 32 input layers,
      # outputting 64 convolutional features, with a square kernel size of 3
      self.conv2 = nn.Conv2d(32, 64, 3, 1)

      # Designed to ensure that adjacent pixels are either all 0s or all active
      # with an input probability
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # First fully connected layer
      self.fc1 = nn.Linear(9216, 128)
      # Second fully connected layer that outputs our 10 labels
      self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)